In [1]:
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from dotenv import load_dotenv
import os
from deep_research import Deep_research_engine
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dataclasses import dataclass


In [2]:
load_dotenv()
tavily_key=os.getenv('tavily_key')
google_api_key=os.getenv('google_api_key')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))

deepsearch=Deep_research_engine()

@dataclass
class Deps:

    deep_search_result:str
    quick_search_result:str
    edited_deep_search_result:str

agent=Agent(llm, system_prompt="you are a research assistant, you are given a query, decide if you need to do a deep research or a quick search, if you need to do a deep research, use the deep_research_agent tool, if you need to do a quick search, use the search_agent tool, if you need to edit the deep search result, use the editor_agent tool")

@agent.tool
async def deep_research_agent(ctx:RunContext[Deps], query:str):
    """
    This function is used to do a deep research on the web for information on a complex query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await deepsearch.chat(query)
    ctx.deps.deep_search_result=res
    return res

quick_search_agent=Agent(llm,tools=[tavily_search_tool(tavily_key)])
@agent.tool
async def search_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to do a quick search on the web for information on a given query.
    Args:
        query (str): The query to search for
    Returns:
        str: The result of the search
    """
    res=await quick_search_agent.run(query)
    ctx.deps.quick_search_result=res.data
    return res.data
    
editor_agent=Agent(llm, system_prompt="you are an editor, you are given a query, a deep search result, and maybe a quick search result (optional), you need to edit the deep search result to make it more accurate following the query's instructions")

@agent.tool
async def editor_agent(ctx: RunContext[Deps], query:str):
    """
    This function is used to edit the deep search result to make it more accurate following the query's instructions.
    Args:
        query (str): The query containing instructions for editing the deep search result
    Returns:
        str: The edited and improved deep search result
    """
    res=await editor_agent.run(f'query:{query}, deep_search_result:{ctx.deps.deep_search_result}, quick_search_result:{ctx.deps.quick_search_result}')
    ctx.deps.edited_deep_search_result=res
    return res



In [3]:
deps=Deps( deep_search_result="", quick_search_result="",edited_deep_search_result="")

In [4]:
await agent.run('do a quick research on the topic of the moon landing',deps=deps)

AgentRunResult(data='The first human-made object to touch the Moon was Luna 2 in 1959. In 1969 Apollo 11 was the first crewed mission to land on the Moon. There were six crewed landings between 1969 and 1972, and numerous uncrewed landings.\n')